In [100]:
# default imports
import numpy as np
import cv2
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [15, 10]

# helper function for drawing rectangles around tracking objects
def drawRectangle(frame, bbox, color):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

# helper function for drawing hint text in videos
def drawText(frame, txt, location, color=(50, 170, 50)):
    cv2.putText(frame, txt, location, cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

# helper class which stores processing video files metadata: file name and template coordinates
class VideoSource:
    def __init__(self, file_name, bound_box):
        self.file_name = file_name
        self.bound_box = bound_box

    def input_file_path(self):
        return f'data/{self.file_name}.mp4'

    def output_file_path(self, tracker):
        return f"results/{self.file_name}_{tracker}.mp4"

# create video sources array
race_car = VideoSource("race_car", (1300, 400, 150, 120))
horse_show = VideoSource('horse_show', (500, 150, 150, 250))
snooker = VideoSource('snooker', (360, 105, 15, 15))
sources = [race_car, horse_show, snooker]
source = sources[1]

# create trackers array
tracker_types = ['MIL','KCF', 'CSRT']
tracker_type = tracker_types[0]

#depending on selected tracker string initialise corresponding tracker object
if tracker_type == 'MIL':
    tracker = cv2.legacy.TrackerMIL.create()

if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF.create()

if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

# create video capturing object
video = cv2.VideoCapture(source.input_file_path())

#get first video frame
ret, frame = video.read()

# initialise tracker with first video frame and template object coordinates
ok = tracker.init(frame, source.bound_box)

# get original video dimensions
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# create video writer object for saving modified videos
file_name = source.output_file_path(tracker_type)

# record video with tracked object at 10 frames per second rate using XVID codec https://fourcc.org/xvid/ with original width / height
video_out = cv2.VideoWriter(file_name, cv2.VideoWriter_fourcc(*"XVID"), 10, (width, height))

# tracking loop
while video.isOpened():
    # get next video frame
    success, frame = video.read()
    
    if not success:
        break
    
    # ok indicates that tracked object is found on next frame, bbox - bounding box of object on new frame
    ok, bbox = tracker.update(frame)

    if ok:
        drawRectangle(frame, bbox, (0, 255, 0))
    else:
        drawText(frame, "Tracking failure detected", (80, 140), (0, 0, 255))
    
    drawText(frame, tracker_type + " Tracker", (80, 60))

    # Write frame to video
    video_out.write(frame)

video.release()
video_out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

# Results

Side note: video in XVID format is not displayed in video tag, so need to reencode it to h264 using command:
```

ffmpeg -y -i "results/horse_show_CSRT.mp4" -c:v libx264 "results/horse_show_CSRT_x264.mp4"  -hide_banner -loglevel error

```

In general, based on my tests, CSRT showed best results. 

## CSRT

| Video | Comments |
|-------|----------|
|  <video src="results/race_car_CSRT_x264.mp4" width="640" height="480" controls></video>     |  In this case CSRT did really good. Even when car is turning on the track, size of car is changing, it keeps tracking it.       |
|  <video src="results/horse_show_CSRT_x264.mp4" width="640" height="480" controls></video>     |  CSRT deals with this most complex case also really good. It keeps track of rider even when his size is changing (he needs to bounce on horse to keep balance). Even when horse jumps, rider frame is tracking. However this tracker does not handle occlusion - you can see it on 02:44, but overall result is impressive.       |
|  <video src="results/snooker_CSRT_x264.mp4" width="640" height="480" controls></video>     |  Perfect work from CSRT. It keeps track of really small sized white ball and fails only when translation is switched to other camera.       |


## MIL

| Video | Comments |
|-------|----------|
|  <video src="results/race_car_MIL_x264.mp4" width="640" height="480" controls></video>     |  In this case MIL worked well - it kept tracking car's frame even on track turn and finished with slightly better tracking frame size than CSRT       |
|  <video src="results/horse_show_MIL_x264.mp4" width="640" height="480" controls></video>     |  In this case MIL executed worse than CSRT. It started really well, it also does better job of adjusting frame size to tracked object size, but on 01:02 it completely lost tracked object and started to track incorrect object. (I think this is because of similarity of background colors and rider's suit). What is also bad, is that tracker did not report failure - it actually thinks this is correct object to track.        |
|  <video src="results/snooker_MIL_x264.mp4" width="640" height="480" controls></video>     |  In this case MIL also executed worse than CSRT. It actually tracked movement of white ball quite good, but when translation changes to next camera it again does not report failure, but continued to track incorrect object on 00:51. For case of snooker this is not best approach - better to unbderstand where tracker started failing.        |


## KCF

| Video | Comments |
|-------|----------|
|  <video src="results/race_car_KCF_x264.mp4" width="640" height="480" controls></video>     |   In this case KCF performed worse than CSRT and MIL. It did not handle car turning - right after car started turning it reported tracking failure.       |
|  <video src="results/horse_show_KCF_x264.mp4" width="640" height="480" controls></video>     |  In this case KCF also perfomed the worst. It did not handle horse jumping well - it reported failure, however found rider couple of frames later. On 01:16 it also reported failure becase background and rider suit have very similar colors. After that it never managed to find rider's frame again.         |
|  <video src="results/snooker_KCF_x264.mp4" width="640" height="480" controls></video>     |  Completely failed test from this tracker. It reported failing right after white ball started moving and never managed to find its frame again        |